# DCGAN
- 整体网络架构：

![](images/dcgan.png)

In [9]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

## 加载数据集与batch制作

In [10]:
num_features = 784 # 28*28*1，输入的特征数

# 训练参数.
lr_generator = 0.0002
lr_discriminator = 0.0002
training_steps = 20000
batch_size = 128
display_step = 500

# 初始化随机向量维度
noise_dim = 100 

In [11]:
# 加载数据
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 类型转换
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# 特征归一化
x_train, x_test = x_train / 255., x_test / 255.

In [12]:
# 制作batch数据
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(10000).batch(batch_size).prefetch(1)

## 生成与判别网络架构设计

In [13]:
# 创建生成器模型
# 输入: 随机向量, Output: 生成的图像数据
class Generator(Model):
    # 用到的层
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = layers.Dense(7 * 7 * 128)
        # 防止过拟合
        self.bn1 = layers.BatchNormalization()
        # 反卷积，14x14x64
        self.conv2tr1 = layers.Conv2DTranspose(64, 5, strides=2, padding='SAME')
        self.bn2 = layers.BatchNormalization()
        # 再一次反卷积，28x28x1 
        self.conv2tr2 = layers.Conv2DTranspose(1, 5, strides=2, padding='SAME')

    # 前向传播计算
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.bn1(x, training=is_training)
        x = tf.nn.leaky_relu(x)
        # 转换成4-D图像数据: (batch, height, width, channels)
        # (batch, 7, 7, 128)
        x = tf.reshape(x, shape=[-1, 7, 7, 128])
        # (batch, 14, 14, 64)
        x = self.conv2tr1(x)
        x = self.bn2(x, training=is_training)
        x = tf.nn.leaky_relu(x)
        # 还原成(batch, 28, 28, 1)
        x = self.conv2tr2(x)
        x = tf.nn.tanh(x)
        return x

# 判别器模型
class Discriminator(Model):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = layers.Conv2D(64, 5, strides=2, padding='SAME')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(128, 5, strides=2, padding='SAME')
        self.bn2 = layers.BatchNormalization()
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(1024)
        self.bn3 = layers.BatchNormalization()
        self.fc2 = layers.Dense(2)

    def call(self, x, is_training=False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.bn1(x, training=is_training)
        x = tf.nn.leaky_relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=is_training)
        x = tf.nn.leaky_relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn3(x, training=is_training)
        x = tf.nn.leaky_relu(x)
        return self.fc2(x)

# 创建网络模型
generator = Generator()
discriminator = Discriminator()

## 损失函数定义

In [14]:
# 损失函数，先看一下，在哪里用到generator_loss
def generator_loss(reconstructed_image):
    gen_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        # labels=tf.ones([batch_size]把标签设置成1
        logits=reconstructed_image, labels=tf.ones([batch_size], dtype=tf.int32)))
    return gen_loss
# 判别器，需要把真的预测成真的，把假的预测成假的
def discriminator_loss(disc_fake, disc_real):
    disc_loss_real = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        # 把真的判别成真的，labels=tf.ones([batch_size]
        logits=disc_real, labels=tf.ones([batch_size], dtype=tf.int32)))
    disc_loss_fake = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        # 把假的判别成假的，labels=tf.zeros([batch_size]
        logits=disc_fake, labels=tf.zeros([batch_size], dtype=tf.int32)))
    return disc_loss_real + disc_loss_fake

# 优化器
optimizer_gen = tf.optimizers.Adam(learning_rate=lr_generator)#, beta_1=0.5, beta_2=0.999)
optimizer_disc = tf.optimizers.Adam(learning_rate=lr_discriminator)#, beta_1=0.5, beta_2=0.999)

In [15]:
def run_optimization(real_images):
    
    # 将特征处理成 [-1, 1]
    real_images = real_images * 2. - 1.

    # 随机产生噪音数据
    noise = np.random.normal(-1., 1., size=[batch_size, noise_dim]).astype(np.float32)
    
    with tf.GradientTape() as g:
        # 噪音数据，传入到生成器
        fake_images = generator(noise, is_training=True)
        # 分别把假的和真的，都输入到判别器，看看判别结果
        disc_fake = discriminator(fake_images, is_training=True)
        disc_real = discriminator(real_images, is_training=True)

        disc_loss = discriminator_loss(disc_fake, disc_real)
            
    # 判别器优化，根据损失值，求梯度
    gradients_disc = g.gradient(disc_loss,  discriminator.trainable_variables)
    # 梯度和对应的参数，进行更新
    optimizer_disc.apply_gradients(zip(gradients_disc,  discriminator.trainable_variables))
    
    # 随机产生噪音数据
    noise = np.random.normal(-1., 1., size=[batch_size, noise_dim]).astype(np.float32)
    
    with tf.GradientTape() as g:
            
        fake_images = generator(noise, is_training=True)
        # 我们希望，判别器，判断出的结果是1，disc_fake=1
        disc_fake = discriminator(fake_images, is_training=True)
        # 计算生成器的损失
        gen_loss = generator_loss(disc_fake)
    # 生成器优化        
    gradients_gen = g.gradient(gen_loss, generator.trainable_variables)
    optimizer_gen.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    
    return gen_loss, disc_loss

## 训练模型

In [16]:
import datetime
# 迭代
time1 = datetime.datetime.now()

for step, (batch_x, _) in enumerate(train_data.take(training_steps + 1)):
    
    if step == 0:
        # 初始化随机向量
        noise = np.random.normal(-1., 1., size=[batch_size, noise_dim]).astype(np.float32)
        # 计算初始损失
        gen_loss = generator_loss(discriminator(generator(noise)))
        disc_loss = discriminator_loss(discriminator(batch_x), discriminator(generator(noise)))
        print("initial: gen_loss: %f, disc_loss: %f" % (gen_loss, disc_loss))
        continue
    
    # 训练
    gen_loss, disc_loss = run_optimization(batch_x)
    
    if step % display_step == 0:
        print("step: %i, gen_loss: %f, disc_loss: %f" % (step, gen_loss, disc_loss))
        
print(datetime.timedelta(seconds = datetime.datetime.now() - time1))

initial: gen_loss: 0.689955, disc_loss: 1.410523
step: 500, gen_loss: 2.163861, disc_loss: 0.477877
step: 1000, gen_loss: 2.202099, disc_loss: 0.515468
step: 1500, gen_loss: 2.192609, disc_loss: 0.455077
step: 2000, gen_loss: 2.780615, disc_loss: 0.263023
step: 2500, gen_loss: 3.157146, disc_loss: 0.275691
step: 3000, gen_loss: 3.242598, disc_loss: 0.309788
step: 3500, gen_loss: 2.930309, disc_loss: 0.399308
step: 4000, gen_loss: 3.177963, disc_loss: 0.296672
step: 4500, gen_loss: 3.652167, disc_loss: 0.250366
step: 5000, gen_loss: 3.578712, disc_loss: 0.213626
step: 5500, gen_loss: 3.860200, disc_loss: 0.135837
step: 6000, gen_loss: 3.983262, disc_loss: 0.239396
step: 6500, gen_loss: 4.296060, disc_loss: 0.147409
step: 7000, gen_loss: 3.893850, disc_loss: 0.087007
step: 7500, gen_loss: 4.520547, disc_loss: 0.141732
step: 8000, gen_loss: 4.684979, disc_loss: 0.156825
step: 8500, gen_loss: 3.610933, disc_loss: 0.165318


KeyboardInterrupt: 

In [ ]:
# 结果展示
import matplotlib.pyplot as plt
plt.style.use('classic')
n = 6
canvas = np.empty((28 * n, 28 * n))
for i in range(n):
    # 还是随机输入
    z = np.random.normal(-1., 1., size=[n, noise_dim]).astype(np.float32)
    # 生成结果
    g = generator(z).numpy()
    # 还原成[0, 1]
    g = (g + 1.) / 2
    g = -1 * (g - 1)
    for j in range(n):
        canvas[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

plt.figure(figsize=(n, n))
plt.imshow(canvas, origin="upper", cmap="gray")
plt.show()

# 对抗生成网络，代表算法

- GAN [论文地址](https://arxiv.org/abs/1406.2661)  [代码地址](https://github.com/goodfeli/adversarial)
- DCGAN [论文地址](https://arxiv.org/abs/1511.06434)  [代码地址](https://github.com/floydhub/dcgan)
- CGAN [论文地址](https://arxiv.org/abs/1411.1784)  [代码地址](https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras)
- CycleGAN [论文地址](https://arxiv.org/abs/1703.10593v6)  [代码地址](https://github.com/junyanz/CycleGAN)
- CoGAN [论文地址](https://arxiv.org/abs/1606.07536)  [代码地址](https://github.com/mingyuliutw/CoGAN)
- ProGAN [论文地址](https://arxiv.org/abs/1710.10196)  [代码地址](https://github.com/tkarras/progressive_growing_of_gans)
- WGAN [论文地址](https://arxiv.org/abs/1701.07875v3)  [代码地址](https://github.com/eriklindernoren/Keras-GAN)
- SAGAN [论文地址](https://arxiv.org/abs/1805.08318v1)  [代码地址](https://github.com/heykeetae/Self-Attention-GAN)
- BigGAN [论文地址](https://arxiv.org/abs/1809.11096v2)  [代码地址](https://github.com/huggingface/pytorch-pretrained-BigGAN)